# Element Load Transformation

## Beam load

This document describes the process of transforming element loads, such as **uniformly distributed loads (UDL)** and **point loads**, from the **global coordinate system** to the **local coordinate system**.

In [1]:
import opstool as opst
import openseespy.opensees as ops

### 2D Case

First we create three 2D beam elements:

In [2]:
ops.wipe()
ops.model("basic", "-ndm", 2, "-ndf", 3)
ops.node(1, 0, 0)
ops.node(2, 0, 2)
ops.node(3, 2, 2)

ops.geomTransf("Linear", 1)
ops.element("elasticBeamColumn", 1, 1, 2, 1000, 10000, 10000, 1)
ops.element("elasticBeamColumn", 2, 2, 3, 1000, 10000, 10000, 1)
ops.element("elasticBeamColumn", 3, 1, 3, 1000, 10000, 10000, 1)

Then, the time series and load pattern are created, followed by the generation of beam element loads using two functions that can easily transform the loads in the global coordinate system to the local coordinate system of each beam element and generate the loads using the [EleLoad Command](https://opensees.berkeley.edu/wiki/index.php/EleLoad_Command) in OpenSees.

In [3]:
ops.timeSeries("Linear", 1)
ops.pattern("Plain", 1, 1)

opst.pre.transform_beam_uniform_load([1, 2, 3], wy=-2)

ops.pattern("Plain", 2, 1)
opst.pre.transform_beam_point_load([1, 2, 3], py=-3, xl=0.5)

We can check this visually. We can see that our loads are along the global Y axis and they are correctly transformed to each beam element according to their local axes.

In [4]:
fig = opst.vis.plotly.plot_model(
    show_ele_loads=True, show_nodal_loads=True, load_scale=2
)
# fig.show()

OPSTOOL ::  Model data has been saved to _OPSTOOL_ODB/ModelData-None.nc!

In [5]:
fig.write_html("images/beamLoad2D_plotly.html", full_html=False, include_plotlyjs="cdn")

### 3D Case

In [6]:
ops.wipe()
ops.model("basic", "-ndm", 3, "-ndf", 6)
ops.node(1, 0, 0, 0)
ops.node(2, 0, 2, 0)
ops.node(3, 2, 2, 0)
ops.node(4, 2, 0, 0)


ops.geomTransf("Linear", 1, 0, 0, 1)
ops.element("elasticBeamColumn", 1, 1, 2, 1000, 1000, 1000, 1000, 1000, 1000, 1)
ops.element("elasticBeamColumn", 2, 2, 3, 1000, 1000, 1000, 1000, 1000, 1000, 1)
ops.element("elasticBeamColumn", 3, 3, 4, 1000, 1000, 1000, 1000, 1000, 1000, 1)
ops.element("elasticBeamColumn", 4, 4, 1, 1000, 1000, 1000, 1000, 1000, 1000, 1)
ops.element("elasticBeamColumn", 5, 1, 3, 1000, 1000, 1000, 1000, 1000, 1000, 1)

In [7]:
ops.timeSeries("Linear", 1)
ops.pattern("Plain", 1, 1)

opst.pre.transform_beam_uniform_load([1, 2, 3, 4, 5], wy=2, wz=-2)

ops.pattern("Plain", 2, 1)
opst.pre.transform_beam_point_load([1, 2, 3, 4, 5], py=2, pz=-3, xl=0.5)

We can check this visually and see that our loads are correctly transformed into the local axes of each beam element.

In [8]:
fig = opst.vis.plotly.plot_model(
    show_ele_loads=True, show_nodal_loads=True, load_scale=2, show_local_axes=True
)
# fig.show()

OPSTOOL ::  Model data has been saved to _OPSTOOL_ODB/ModelData-None.nc!

In [9]:
fig.write_html("images/beamLoad3D_plotly.html", full_html=False, include_plotlyjs="cdn")

## Surface Load

According to the static equivalence principle, the surface distributed load is equivalent to the node load.

In [10]:
import opstool as opst
import openseespy.opensees as ops

In [11]:
ops.wipe()
# set up a 3D-6DOFs model
ops.model("Basic", "-ndm", 3, "-ndf", 6)
ops.node(1, 0.0, 0.0, 0.0)
ops.node(2, 1.0, 0.0, 0.0)
ops.node(3, 1.0, 1.0, 0.0)
ops.node(4, 0.0, 1.0, 0.0)

ops.node(5, 2.0, 0.0, 0.0)
ops.node(6, 2.0, 1.0, 0.0)


# create a fiber shell section with 4 layers of material 1
# each layer has a thickness = 0.025
ops.nDMaterial("ElasticIsotropic", 1, 1000.0, 0.2)
ops.section("LayeredShell", 11, 4, 1, 0.025, 1, 0.025, 1, 0.025, 1, 0.025)
# create the shell element using the small displacements/rotations assumption
ops.element("ASDShellQ4", 1, 1, 2, 3, 4, 11)
ops.element("ASDShellT3", 2, 2, 5, 6, 11)
ops.element("ASDShellT3", 3, 6, 3, 2, 11)

Using ASDShellQ4 - Developed by: Massimo Petracca, Guido Camata, ASDEA Software Technology
Using ASDShellT3 - Developed by: Massimo Petracca, Guido Camata, ASDEA Software Technology


In [12]:
ops.timeSeries("Linear", 1)
ops.pattern("Plain", 1, 1)

opst.pre.transform_surface_uniform_load(ele_tags=[1, 2, 3], p=-1)

Since the surface load intensity is 1 and the total area is 2, the total load on the three elements is 2.0. The sum of the nodal loads assigned to all nodes should equal 2.0.

In [13]:
fig = opst.vis.plotly.plot_model(
    show_nodal_loads=True, load_scale=2, show_local_axes=True
)
# fig.show()

OPSTOOL ::  Model data has been saved to _OPSTOOL_ODB/ModelData-None.nc!

In [14]:
fig.write_html(
    "images/surfaceLoad3D_plotly.html", full_html=False, include_plotlyjs="cdn"
)